Reading file

In [5]:
import numpy as np
import pandas as pd

In [6]:
taste = pd.read_csv('taste_profiles_V8.csv')
taste.head()

,Unnamed: 0.1,Unnamed: 0,Product_description,Key_ingredient,Food_group,m_sweet,m_sour,m_bitter,m_umami,m_salt,m_fat,fat_100g,carbs_100g,proteins_100g
0,0,0,Chips pre-fried unprepared,chips,Potatoes,9,3,0,7,15,42,5.6,0.6,7.1
1,1,1,Potatoes boiled with skin av,potato,Potatoes,6,2,1,8,6,9,35.0,54.0,5.0
2,2,2,Crackers cream,cracker,Bread,5,1,1,1,8,7,12.0,76.0,8.0
3,3,3,Bread Tijger white,bread,Bread,7,2,1,0,13,12,11.0,30.0,9.0
4,4,4,Marmite,marmite,Miscellaneous,11,23,23,64,62,19,17.0,49.0,16.0


In [7]:
rp = pd.read_csv('cleaned_recipesv2.csv')
rp.head()

,recipeID,title,ingredients,directions,link,source,NER
0,5,Cheeseburger Potato Soup,"[""6 baking potatoes"", ""1 lb. of extra lean gro...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,Gathered,"[""baking potatoes"", ""extra lean ground beef"", ..."
1,40,Chicken Stew,"[""3 lb. chicken, boiled"", ""4 medium potatoes, ...","[""Remove chicken from bone."", ""Use the broth.""...",www.cookbooks.com/Recipe-Details.aspx?id=284237,Gathered,"[""chicken"", ""potatoes"", ""onions"", ""corn"", ""Eng..."
2,44,Fruit Pizza,"[""1 box yellow cake mix"", ""1/4 c. water"", ""1/4...","[""Mix ingredients and spread into 2 greased pi...",www.cookbooks.com/Recipe-Details.aspx?id=28243,Gathered,"[""yellow cake mix"", ""water"", ""brown sugar"", ""b..."
3,48,Mexican Cookie Rings,"[""1 1/2 c. sifted flour"", ""1/2 tsp. baking pow...","[""Sift flour, baking powder and salt together....",www.cookbooks.com/Recipe-Details.aspx?id=364136,Gathered,"[""flour"", ""baking powder"", ""salt"", ""butter"", ""..."
4,71,Potato Casserole,"[""6 c. diced potatoes"", ""1/2 c. chopped onion""...","[""Peel and dice potatoes; place in bowl of col...",www.cookbooks.com/Recipe-Details.aspx?id=261749,Gathered,"[""potatoes"", ""onion"", ""butter"", ""cream of chic..."


In [8]:
rping = pd.read_csv('recipe_ingredientsv6.csv')
rping.head()

,recipeID,quantity,unit,ingredient_NER,ingredient_Key_ingredient,rest of ingredient,metric_qty
0,5,1.00,cups,buttermilk,buttermilk,buttermilk,0.236
1,5,1.00,cups,buttermilk,buttermilk,buttermilk,0.236
2,40,0.25,cups,onion,onion,chopped onion (or as much as you want),0.059
3,40,1.00,packages,broccoli,broccoli,chopped broccoli,0.500
4,44,4.00,cups,cabbage,cabbage,shredded cabbage,0.944


RDF  
Step 1) Finding all triples: subject, predicate, object  
Step 2) Making a predicate: recipeA is a satisfying substitute for recipeB.   
                            Satisfying nutritions + flavors  
Step 3) Visualizing the graph  

```
Questions
namespace example? 
RDF scheme?
Product_description is to be deleted? (Yes)
BNode vs URIRef? (BNode probably, But confirm it. I should use BNode)
Getting the updated file and replace the column name (Done)
For defining predicates, namespace needed?


taste
    Product_description hasKeyIngredient Key_ingredient 
    Key_ingredient hasSweetness m_sweet
    Key_ingredient hasSourness m_sour
    Key_ingredient hasBitterness m_bitter
    Key_ingredient hasUmaminess m_umami
    Key_ingredient hasSaltness m_salf
    Key_ingredient hasfatness m_fat
    (Product_description hasSweetness m_sweet)
    (Automatic...)
    Key_ingredient hasCarb carb
    Key_ingredient hasProtein protein
    Key_ingredient hasFat fat

recipe + recipe_ingredients
    title hasID Unnamed:0 (Vise versa?)
    recipeID is Unnamed:0 (change Unnamed:0 to recipeID)
    title hasNERIngredient ingredient_NER (Not needed?)
    title hasKeyIngredient ingredient_Key_ingredient
    (rest of ingredient not necessary?)
    ingredient_NER hasQuantity quantity
    ingredient_Key_ingredient hasQuantity quantity
    ingredient_NER hasUnit unit
    ingredient_Key_ingredient hasQuantity quantity
    # How to deal with specific recipe has diff quantity, unit?
    # define property to object or include property in object 
    recipe_ID hasIngredient ingredient_Key_ingredient

taste + recipe + recipe_ingredients
    Product's key ingredient is the ingredient 

```

In [23]:
from rdflib import Graph, Literal, BNode, Namespace, RDF, RDFS, XSD

n = Namespace("http://example.org/")
g = Graph()

# define classes and properties
g.add((n.Recipe, RDF.type, RDFS.Class))
g.add((n.Ingredient, RDF.type, RDFS.Class))
g.add((n.hasIngredient, RDF.type, RDF.Property))
g.add((n.hasIngredient, RDFS.domain, n.Recipe))
g.add((n.hasIngredient, RDFS.range, n.Ingredient))
g.add((n.hasRecipeName, RDF.type, RDF.Property))
g.add((n.hasRecipeName, RDFS.domain, n.Recipe))
g.add((n.hasRecipeName, RDFS.range, n.Recipe))
g.add((n.hasRecipeID, RDF.type, RDF.Property))
g.add((n.hasRecipeID, RDFS.domain, n.Recipe))
g.add((n.hasRecipeID, RDFS.range, XSD.integer))
g.add((n.hasIngredientName, RDF.type, RDF.Property))
g.add((n.hasIngredientName, RDFS.domain, n.Ingredient))
g.add((n.hasIngredientName, RDFS.range, XSD.string))

# define quantity
g.add((n.hasQuantityOf, RDF.type, RDF.Property))
g.add((n.hasQuantityOf, RDFS.domain, n.Ingredient))
g.add((n.hasQuantityOf, RDFS.range, XSD.integer))

# define flavor properties
g.add((n.hasSweetness, RDF.type, RDF.Property))
g.add((n.hasSweetness, RDFS.domain, n.Ingredient))
g.add((n.hasSweetness, RDFS.range, XSD.integer))
g.add((n.hasSourness, RDF.type, RDF.Property))
g.add((n.hasSourness, RDFS.domain, n.Ingredient))
g.add((n.hasSourness, RDFS.range, XSD.integer))
g.add((n.hasBitterness, RDF.type, RDF.Property))
g.add((n.hasBitterness, RDFS.domain, n.Ingredient))
g.add((n.hasBitterness, RDFS.range, XSD.integer))
g.add((n.hasUmaminess, RDF.type, RDF.Property))
g.add((n.hasUmaminess, RDFS.domain, n.Ingredient))
g.add((n.hasUmaminess, RDFS.range, XSD.integer))
g.add((n.hasSaltness, RDF.type, RDF.Property))
g.add((n.hasSaltness, RDFS.domain, n.Ingredient))
g.add((n.hasSaltness, RDFS.range, XSD.integer))
g.add((n.hasFatness, RDF.type, RDF.Property))
g.add((n.hasFatness, RDFS.domain, n.Ingredient))
g.add((n.hasFatness, RDFS.range, XSD.integer))

# define nutrition properties
g.add((n.hasCarb, RDF.type, RDF.Property))
g.add((n.hasCarb, RDFS.domain, n.Ingredient))
g.add((n.hasCarb, RDFS.range, XSD.integer))
g.add((n.hasProtein, RDF.type, RDF.Property))
g.add((n.hasProtein, RDFS.domain, n.Ingredient))
g.add((n.hasProtein, RDFS.range, XSD.integer))
g.add((n.hasFat, RDF.type, RDF.Property))
g.add((n.hasFat, RDFS.domain, n.Ingredient))
g.add((n.hasFat, RDFS.range, XSD.integer))

<Graph identifier=Ndbb513410f5a4d26ba0f5371a9ce3608 (<class 'rdflib.graph.Graph'>)>

In [24]:
for (_, t_row), (_, rp_row), (_, rping_row) in zip(taste.iterrows(), rp.iterrows(), rping.iterrows()):
    recipe = BNode()
    ingredient = BNode()
    # ingredient = n[str(row['Key_ingredient'])]
    # recipe = n[str(row['title'])]
    
    g.add((recipe, RDF.type, n.Recipe))
    g.add((ingredient, RDF.type, n.Ingredient))
    g.add((recipe, n.hasIngredient, ingredient))
        
    g.add((recipe, n.hasRecipeName, Literal(rp_row['title'])))
    g.add((recipe, n.hasRecipeID, Literal(rp_row['recipeID'], datatype=XSD.integer)))
    g.add((ingredient, n.hasIngredientName, Literal(rping_row['ingredient_Key_ingredient'])))
    
    g.add((ingredient, n.hasQuantityOf, Literal(rping_row['metric_qty'], datatype=XSD.float)))
    
    g.add((ingredient, n.hasSweetness, Literal(t_row['m_sweet'], datatype=XSD.integer)))
    g.add((ingredient, n.hasSourness, Literal(t_row['m_sour'], datatype=XSD.integer)))
    g.add((ingredient, n.hasBitterness, Literal(t_row['m_bitter'], datatype=XSD.integer)))
    g.add((ingredient, n.hasUmaminess, Literal(t_row['m_umami'], datatype=XSD.integer)))
    g.add((ingredient, n.hasSaltness, Literal(t_row['m_salt'], datatype=XSD.integer)))
    g.add((ingredient, n.hasFatness, Literal(t_row['m_fat'], datatype=XSD.integer)))
    
    g.add((ingredient, n.hasCarb, Literal(t_row['carbs_100g'], datatype=XSD.float)))
    g.add((ingredient, n.hasProtein, Literal(t_row['proteins_100g'], datatype=XSD.float)))
    g.add((ingredient, n.hasFat, Literal(t_row['fat_100g'], datatype=XSD.float)))
    
filename = 'output.ttl'
serialized_graph = g.serialize(format='turtle')
with open(filename, 'wb') as f:
    f.write(serialized_graph.encode('utf-8'))

In [27]:
query = """
    SELECT ?recipeName ?ingredientName ?sweetness
    WHERE {
        ?recipe n:hasRecipeName ?recipeName .
        ?ingredient n:hasIngredientName ?ingredientName .
        ?ingredient n:hasSweetness ?sweetness .
    }
"""

result = g.query(query, initNs={'n': n})
for row in result:
    print(f"Recipe {row[0]} has ingredient {row[1]} and its sweetness is {row[2]}")

Recipe Cheeseburger Potato Soup has ingredient buttermilk and its sweetness is 9
Recipe Chicken Stew has ingredient buttermilk and its sweetness is 9
Recipe Fruit Pizza has ingredient buttermilk and its sweetness is 9
Recipe Mexican Cookie Rings has ingredient buttermilk and its sweetness is 9
Recipe Potato Casserole has ingredient buttermilk and its sweetness is 9
Recipe Fast Real Good Fudge has ingredient buttermilk and its sweetness is 9
Recipe Ambrosia has ingredient buttermilk and its sweetness is 9
Recipe Red Cabbage And Apples has ingredient buttermilk and its sweetness is 9
Recipe Ranch Beef And Beans has ingredient buttermilk and its sweetness is 9
Recipe Cheese Ball has ingredient buttermilk and its sweetness is 9
Recipe Cheese Ball has ingredient buttermilk and its sweetness is 9
Recipe Cucumber Salad With Dill has ingredient buttermilk and its sweetness is 9
Recipe Chicken Pot Pie has ingredient buttermilk and its sweetness is 9
Recipe Peanut Butter Cup Cookies has ingredie

```SPARQL 
Visualization

query: 
finding ingredients that have 10% carb, 70% fat... 
finding ingredients that have the similar(same) flavor as the pre-existing recipe
```